In [24]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

def is_name(word):
    letters = tuple(filter(lambda char: char.isalpha(), word))
    if not letters:
        return False
    return letters[0].isupper() and ''.join(letters[1:]).islower()

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if not bool(word) or word[0].isdigit():
            # filter wrong words
            continue

        if is_name(word):
            print "ok::%s\t%d" % (word.lower(), 1)
        else:
            print "not::%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [21]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer.py


In [22]:
%%writefile reducer_2.py

import sys

current_word = None
word_sum_ok = 0
word_sum_not = 0

PERCENT = 0.5

def print_name_precentage(word, correct, incorrect):
    if not (correct + incorrect):
        print >> sys.stderr, "reporter:counter:Name stats,zero,%d" % 1
    elif (100. * incorrect / (correct + incorrect)) < PERCENT:
        print "%s\t%d" % (word, correct + incorrect)
        print >> sys.stderr, "reporter:counter:Name stats,name,%d" % 1
    else:
        print >> sys.stderr, "reporter:counter:Name stats,word,%d" % 1

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        flag, word = key.split('::', 1)
        count = int(count)
    except ValueError as e:
        continue

    if current_word != word:
        print_name_precentage(current_word, word_sum_ok, word_sum_not)
        word_sum_ok = 0
        word_sum_not = 0
        current_word = word        

    if flag == 'ok':
        word_sum_ok += count
    elif flag == 'not':
        word_sum_not += count

if current_word:
    print_name_precentage(current_word, word_sum_ok, word_sum_not)


Overwriting reducer_2.py


In [28]:
%%bash

NUM_REDUCERS=8

IN_DIR="/data/wiki/en_articles_part"
TMP_DIR="namecount_tmp" # $(date +"%s%6N")
OUT_DIR="namecount_res" # $(date +"%s%6N")

hdfs dfs -rm -r -f -skipTrash ${TMP_DIR} > /dev/null
hdfs dfs -rm -r -f -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming name count prepare" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator='::' \
    -D mapreduce.partition.keycomparator.options='-k2' \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input ${IN_DIR} \
    -output ${TMP_DIR} > /dev/null

NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming name count sort result" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.map.output.key.field.separator='::' \
    -D mapreduce.partition.keycomparator.options='-k2' \
    -files reducer_2.py \
    -mapper "cat" \
    -reducer "python reducer_2.py" \
    -input ${TMP_DIR} \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -text ${OUT_DIR}/part-00000 | sort -k2rn | head -5 | tail -1

french	5754


17/10/17 11:24:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/17 11:24:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/10/17 11:24:15 INFO mapred.FileInputFormat: Total input files to process : 1
17/10/17 11:24:15 INFO mapreduce.JobSubmitter: number of splits:2
17/10/17 11:24:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1508231375397_0028
17/10/17 11:24:16 INFO impl.YarnClientImpl: Submitted application application_1508231375397_0028
17/10/17 11:24:16 INFO mapreduce.Job: The url to track the job: http://843f47db8d38:8088/proxy/application_1508231375397_0028/
17/10/17 11:24:16 INFO mapreduce.Job: Running job: job_1508231375397_0028
17/10/17 11:24:23 INFO mapreduce.Job: Job job_1508231375397_0028 running in uber mode : false
17/10/17 11:24:23 INFO mapreduce.Job:  map 0% reduce 0%
17/10/17 11:24:42 INFO mapreduce.Job:  map 12% reduce 0%
17/10/17 11:24:48 INFO mapreduce.Job:  map 18% reduce 0%
17/10/17 11:24:54 INFO 